In [193]:
import pandas as pd
from subprocess import run, PIPE
from pathlib import Path
from joblib import Parallel, delayed
import numpy as np
from pickle import dump

In [136]:
gene_expresson = pd.read_csv("../data/57epigenomes.RPKM.pc", delimiter="\t", index_col=False)
epigenomes = gene_expresson.columns[2:]
histones = ["H3K27me3", "H3K36me3", "H3K4me1", "H3K4me3", "H3K9me3"]

In [137]:
NTHREADS = 16


# Convert to bam

In [27]:
def do(filename):
    result = run(
        f"bedtools bedtobam -i ../data/{filename}.bed -g ../data/hg19chrom.sizes".split(), 
        stdout=PIPE, stderr=PIPE
    )
    if result.returncode != 0:
        print(f"Command fail")
        print(result.stdout, result.stderr)

    with open(f"../data/{filename}.bam", "wb") as f:
        f.write(result.stdout)

filenames = []
for epigenome in epigenomes:
    for histone in histones:
        filename = f"{epigenome}-{histone}"
#         print(f"Current: {filename}")
        if Path(f"../data/{filename}.bam").is_file():
#             print(f"{filename} exists, skip")
            continue
        filenames.append(filename)
        
_ = Parallel(n_jobs=NTHREADS)(delayed(do)(filename) for filename in filenames)


# Sort

In [25]:
%%time
def do(filename):
    result = run(
        f"samtools sort ../data/{filename}.bam ../data/{filename}.sort".split(), 
        stdout=PIPE, stderr=PIPE
    )
filenames = []
for epigenome in epigenomes:
    for histone in histones:
        filename = f"{epigenome}-{histone}"
        if Path(f"../data/{filename}.sort.bam").is_file():
            continue
        filenames.append(filename)
_ = Parallel(n_jobs=NTHREADS)(delayed(do)(filename) for filename in filenames)

CPU times: user 36 ms, sys: 316 ms, total: 352 ms
Wall time: 516 ms


# Index

In [24]:
def do(filename):
    result = run(
        f"samtools index ../data/{filename}.sort.bam".split(), 
        stdout=PIPE, stderr=PIPE
    )
filenames = []
for epigenome in epigenomes:
    for histone in histones:
        filename = f"{epigenome}-{histone}"
        if Path(f"../data/{filename}.sort.bam.bai").is_file():
            continue
        filenames.append(filename)
_ = Parallel(n_jobs=NTHREADS)(delayed(do)(filename) for filename in filenames)

# Generate X

In [140]:
def do(files, epigenome):
    result = run(
        f"bedtools multicov -bams {files} -bed ../data/interest.bed".split(), 
        stdout=PIPE, stderr=PIPE
    )
    if result.returncode != 0:
        print(f"Command fail")
        print(result.stdout, result.stderr)
        
    with open(f"../data/{epigenome}.csv", "wb") as f:
        f.write(result.stdout)
        
filenames = []

for epigenome in epigenomes: 
    pathes = []
    for histone in histones:
        filename = f"{epigenome}-{histone}"
        path = f"../data/{filename}.sort.bam"
        pathes.append(path)

    files = " ".join(pathes)
    if Path(f"../data/{epigenome}.csv").is_file():
        continue
    filenames.append((files, epigenome))

_ = Parallel(n_jobs=NTHREADS)(delayed(do)(files, epigenome) for (files, epigenome) in filenames)

[('../data/E053-H3K27me3.sort.bam ../data/E053-H3K36me3.sort.bam ../data/E053-H3K4me1.sort.bam ../data/E053-H3K4me3.sort.bam ../data/E053-H3K9me3.sort.bam', 'E053'), ('../data/E054-H3K27me3.sort.bam ../data/E054-H3K36me3.sort.bam ../data/E054-H3K4me1.sort.bam ../data/E054-H3K4me3.sort.bam ../data/E054-H3K9me3.sort.bam', 'E054'), ('../data/E055-H3K27me3.sort.bam ../data/E055-H3K36me3.sort.bam ../data/E055-H3K4me1.sort.bam ../data/E055-H3K4me3.sort.bam ../data/E055-H3K9me3.sort.bam', 'E055'), ('../data/E056-H3K27me3.sort.bam ../data/E056-H3K36me3.sort.bam ../data/E056-H3K4me1.sort.bam ../data/E056-H3K4me3.sort.bam ../data/E056-H3K9me3.sort.bam', 'E056'), ('../data/E057-H3K27me3.sort.bam ../data/E057-H3K36me3.sort.bam ../data/E057-H3K4me1.sort.bam ../data/E057-H3K4me3.sort.bam ../data/E057-H3K9me3.sort.bam', 'E057'), ('../data/E058-H3K27me3.sort.bam ../data/E058-H3K36me3.sort.bam ../data/E058-H3K4me1.sort.bam ../data/E058-H3K4me3.sort.bam ../data/E058-H3K9me3.sort.bam', 'E058'), ('../data

KeyboardInterrupt: 

# Generate XY

In [183]:
Xs = []
Ys = []
for epigenome in epigenomes:
    if Path(f"../data/{epigenome}.csv").is_file():
        df = pd.read_csv(f"../data/{epigenome}.csv", delimiter="\t", header=None, index_col=None)
        names = []
        groups = []
        for idx in range(len(df)//100):
            selection = df.iloc[idx*100:idx*100+100]
            names.append(selection[3].iloc[0])
            groups.append(selection[[4,5,6,7,8]].values[None,:,:])
        X = np.concatenate(groups)
        rawY = gene_expresson.set_index("gene_id").loc[names].E003
        Y = (rawY > rawY.median()) + 0
        Xs.append(X)
        Ys.append(Y)
        
Xs = np.concatenate([X[None,:,:,:] for X in Xs])
Ys = np.concatenate([Y[None,:] for Y in Ys])

In [194]:
with open("../data/input.pkl", "wb") as f:
    dump((Xs, Ys), f)